# Tangibles for Week 1 Blackjack Simulation Work
> The current progress on my lesson and the Python version of the simulation (using Objects!)
- toc: true
- title: Week 1 DVASS Tangibles - Drew Reed
- permalink: /dvassweek1drew/
- categories: [python]

## Python Simulation Setup

Below is my progress on the Python blackjack simulation, which can be used as the basis for the rest of the lessons.

### Card Class

I decided to use a class to define the cards and deck, since they'd be easier to interact with using the other functions.

In [18]:
class Card:
    def __init__(self, suit, val):
        self.suit = suit
        self.val = val
        if val == 11:
            self.kind = "Ace"
        elif val == 12:
            self.kind = "Jack"
        elif val == 13:
            self.kind = "Queen"
        elif val == 14:
            self.kind = "King"
        else:
            self.kind = str(self.val)

    def show(self):
        return f"{self.kind} of {self.suit}"
    
    def ace_adj(self):
        if self.kind == "Ace":
            self.val = 1

card = Card("Spades", 11)
card.show()

'Ace of Spades'

### Deck Class

Here is a class that is able to form itself full of cards and shuffle.

In [19]:
import random

class Deck:
    def __init__(self):
        self.cards = []
        self.build()

    def build(self):
        for suit in ["Spades", "Clubs", "Diamonds", "Hearts"]:
            for val in range (2, 15):
                self.cards.append(Card(suit, val))
    
    def show(self):
        card_disp = [card.show() for card in self.cards]
        print(card_disp)
    
    def shuffle(self):
        random.shuffle(self.cards)

    def draw_card(self):
        return self.cards.pop()

deck = Deck()
deck.shuffle()
deck.show()

['3 of Spades', '6 of Hearts', '8 of Diamonds', 'King of Hearts', '4 of Spades', 'Ace of Diamonds', 'Jack of Diamonds', '4 of Hearts', '2 of Diamonds', '7 of Hearts', 'Queen of Diamonds', 'Jack of Spades', '5 of Clubs', '2 of Spades', 'Queen of Hearts', '4 of Clubs', 'King of Spades', 'Queen of Clubs', '9 of Hearts', '8 of Hearts', '7 of Spades', '6 of Clubs', 'Ace of Spades', 'King of Clubs', '7 of Clubs', '10 of Hearts', '5 of Hearts', 'Jack of Hearts', '3 of Diamonds', '5 of Diamonds', '7 of Diamonds', '10 of Clubs', '6 of Spades', 'Jack of Clubs', '3 of Clubs', 'Queen of Spades', '8 of Clubs', '9 of Clubs', 'Ace of Clubs', '8 of Spades', 'Ace of Hearts', '6 of Diamonds', 'King of Diamonds', '9 of Spades', '2 of Hearts', '4 of Diamonds', '10 of Diamonds', '9 of Diamonds', '10 of Spades', '2 of Clubs', '3 of Hearts', '5 of Spades']


### Blackjack Game Procedures

Bringing the deck and the cards together, the code below simulates a game of blackjack.

For simplicity, there is no splitting with pairs or doubling down at the moment. I may add this later.

In [28]:
from IPython.display import clear_output as clr

player_chips = 100

def game_start():
    clr(wait=True)
    global player_hand #initializing hands and deck
    global dealer_hand
    global deck
    global p_bet
    deck = Deck()
    deck.shuffle()
    player_hand = []
    dealer_hand = []
    p_bet = bet(player_chips) #getting the player bet

    print("Initial draws:") #giving the initial draws
    d1 = hit(dealer_hand)
    print(f"The dealer draws: {d1}")
    p1 = hit(player_hand)
    print(f"You receive: {p1}")
    d2 = hit(dealer_hand)
    print("The dealer draws a face-down card...")
    p2 = hit(player_hand)
    print(f"You receive: {p2}")
    if sum(player_hand) == 21: #instant player win on blackjack
        print("WOW! A blackjack! You win!")
        win(p_bet)
    print("--------------------------------")
    print(f"Dealer's hand: {d1}, ???")
    player_turn() #once player turn finishes, the dealer turn occurs

    if player_chips != 0:
        pa = input('Would you like to play again? (Input "y" for yes and "n" for no.)')
        if pa.lower() == "y":
            game_start()
        else:
            print(f"You finished with {str(player_chips)} chips!")
            return
    else:
        print("You lost all of your chips! Better luck next time.")

def bet(chips):
    print(f"Current chips: {chips}")
    b = input(f"How much would you like to bet? (Input an integer {str(player_chips)} or less.)")
    try:
        if int(b) <= chips:
            return int(b)
    except:
        print("Invalid bet.")
        bet(chips)

def sum(hand):
    sm = 0
    for card in hand:
        if card.val > 11:
            sm += 10
        else:
            sm += card.val
    if sm > 21:
        for card in hand:
            if card.val == 11:
                card.ace_adj()
                return sum(hand)
    return sm

def hit(hand):
    res = deck.draw_card()
    if (res.val == 11) and (sum(hand) + 11 > 21): #adjusting ace if it would break
        res.ace_adj()
    hand.append(res)
    return res.show()

def hand_display(hand):
    disp_hand = []
    for card in hand:
        disp_hand.append(card.show())
    return ", ".join(disp_hand)

def player_turn():
    print(f"Your hand: {hand_display(player_hand)}")
    if sum(player_hand) > 21:
        print("You break! You lose.")
        lose(p_bet)
        return
    rsp = input("Would you like to hit (h) or stay (s)? (input either option)")
    if rsp == "h":
        received = hit(player_hand)
        print(f"You drew a {received}!")
        player_turn()
    elif rsp == "s":
        print("You stand.")
        dealer_turn()
    else:
        print('Invalid input. Input "h" to hit or "s" to stand.')
        player_turn()

def dealer_turn():
    print(f"Dealer's hand: {hand_display(dealer_hand)}")
    if sum(dealer_hand) > 16:
        print("The dealer stays.")
        pass
    else:
        print(f"The dealer draws: {hit(dealer_hand)}")
        if sum(dealer_hand) > 21:
            print("The dealer breaks! You win.")
            win(p_bet)
            return
        dealer_turn()
        return
    if sum(player_hand) > sum(dealer_hand):
        print(f"Congratulations! You won with a hand worth {sum(player_hand)}!")
        win(p_bet)
    elif sum(dealer_hand) > sum(player_hand):
        print(f"Too bad! You lost to the dealer's hand, worth {sum(dealer_hand)}.")
        lose(p_bet)
    else:
        print("It's a push! You keep your bet.")
    return

def win(bet):
    global player_chips
    player_chips += bet
    return

def lose(bet):
    global player_chips
    player_chips -= bet
    return

game_start()

Current chips: 50
Initial draws:
The dealer draws: King of Diamonds
You receive: 2 of Clubs
The dealer draws a face-down card...
You receive: 10 of Diamonds
--------------------------------
Dealer's hand: King of Diamonds, ???
Your hand: 2 of Clubs, 10 of Diamonds
You drew a 9 of Hearts!
Your hand: 2 of Clubs, 10 of Diamonds, 9 of Hearts
You stand.
Dealer's hand: King of Diamonds, 7 of Spades
The dealer stays.
Congratulations! You won with a hand worth 21!
You finished with 100 chips!
